# CHAPTER 3 데이터 정제 및 준비

## 목표
- 결측/문자열 오류/타입 오류/중복을 실무적으로 처리한다.
- 정제된 결과를 다음 단계 입력으로 저장한다.


In [1]:
# ============================================
# [공통] 라이브러리 / 경로 / 출력 옵션 세팅
# 이 셀은 모든 챕터 노트북에서 동일하게 사용합니다.
# ============================================

# 1) 수치 계산(NumPy) / 표 데이터(Pandas) 불러오기
import numpy as np
import pandas as pd

# 2) 파일 경로를 운영체제와 무관하게 다루기 위한 Path
from pathlib import Path

# 3) 현재 노트북이 실행되는 폴더를 기준(BASE)으로 데이터 폴더(DATA) 지정
BASE = Path(".").resolve()          # 현재 작업 폴더(절대경로)
DATA = BASE / "data"                # data 폴더 경로

# 4) Pandas 출력 옵션(교육용): 너무 길게 출력되지 않도록 적당히 제한
pd.set_option("display.max_rows", 12)
pd.set_option("display.max_columns", 30)
pd.set_option("display.width", 140)

# 5) 확인 출력
print("BASE:", BASE)
print("DATA exists:", DATA.exists())


BASE: C:\Users\KDA\python_Code
DATA exists: True


## 3.1 원본 로드 + 결측 파악

In [2]:
# [이 셀은 무엇을 하는가?]
# - messy_events.csv를 로드하고 결측 개수/비율을 확인합니다.

df = pd.read_csv(DATA / "messy_events.csv")
display(df.head())

na_cnt = df.isna().sum()
na_ratio = (df.isna().mean() * 100).round(1)

display(pd.DataFrame({"na_cnt": na_cnt, "na_ratio(%)": na_ratio}))

,date,user,score,city
0,2024-01-06T00:00:00.000000000,Bob,NaN,Incheon
1,2024-01-04T00:00:00.000000000,CHARLIE,15.0,Busan
2,2024-01-01T00:00:00.000000000,NaN,NaN,seoul
3,2024-01-19T00:00:00.000000000,alice,30.0,Seoul
4,2024-01-02T00:00:00.000000000,NaN,NaN,NaN


,na_cnt,na_ratio(%)
date,0,0.0
user,9,13.8
score,31,47.7
city,15,23.1


## 3.2 score를 숫자로 변환(실무에서 가장 흔한 문제)

In [3]:
# [이 셀은 무엇을 하는가?]
# - score 컬럼을 정리(공백 제거/결측 치환)하고 숫자로 변환합니다.

df2 = df.copy()

# 1) 문자열 정리: strip으로 공백 제거
score_str = df2["score"].astype(str).str.strip()

# 2) 비정상 값을 결측으로 치환
score_str = score_str.replace({"": np.nan, "None": np.nan, "N/A": np.nan})

# 3) 숫자 변환(실패는 NaN)
df2["score"] = pd.to_numeric(score_str, errors="coerce")

display(df2.head())
print(df2.dtypes)

,date,user,score,city
0,2024-01-06T00:00:00.000000000,Bob,NaN,Incheon
1,2024-01-04T00:00:00.000000000,CHARLIE,15.0,Busan
2,2024-01-01T00:00:00.000000000,NaN,NaN,seoul
3,2024-01-19T00:00:00.000000000,alice,30.0,Seoul
4,2024-01-02T00:00:00.000000000,NaN,NaN,NaN


date      object
user      object
score    float64
city      object
dtype: object


## 3.3 user/city 표준화(공백/대소문자 통일)

In [4]:
# [이 셀은 무엇을 하는가?]
# - user/city를 표준화하여 같은 값이 다른 문자열로 분리되는 문제를 해결합니다.

df2["user_clean"] = df2["user"].astype(str).str.strip().str.lower().replace({"none": np.nan})
df2["city_clean"] = df2["city"].astype(str).str.strip().str.title().replace({"None": np.nan})

display(df2[["user","user_clean","city","city_clean","score"]].head(12))

,user,user_clean,city,city_clean,score
0,Bob,bob,Incheon,Incheon,NaN
1,CHARLIE,charlie,Busan,Busan,15.0
2,NaN,nan,seoul,Seoul,NaN
3,alice,alice,Seoul,Seoul,30.0
4,NaN,nan,NaN,Nan,NaN
5,NaN,nan,seoul,Seoul,20.0
6,dave,dave,Incheon,Incheon,10.0
7,Bob,bob,seoul,Seoul,NaN
8,CHARLIE,charlie,Seoul,Seoul,15.0
9,alice,alice,Incheon,Incheon,NaN


## 3.4 중복 제거 + 저장

In [5]:
# [이 셀은 무엇을 하는가?]
# - 완전 중복을 제거하고, 정제 결과를 events_clean.csv로 저장합니다.

before = len(df2) # 중복 전에 길이 구하기 
df3 = df2.drop_duplicates() 
after = len(df3)

print("duplicates removed:", before - after)

out_path = DATA / "events_clean.csv"
df3.to_csv(out_path, index=False) # 인덱스 값이 붙여있냐 없냐에 따라 결과 값이 달리지긴함  # 내보낼 때 : to_csv
print("saved:", out_path) 

duplicates removed: 5
saved: C:\Users\KDA\python_Code\data\events_clean.csv


In [6]:
a= [1,2,3,4]
num = len(a)
num

4